In [1]:
from pymongo import MongoClient
from datetime import datetime
import redis
import time
import pickle
import psycopg2
import json


In [2]:
# Connect to postgres database

connection = psycopg2.connect(
    host="db",
    database="testing",
    user="postgres",
    password="example"
)
postgres = connection.cursor()

(2, 'extension2', {'current': []})


In [6]:
# Connect to redis

pool = redis.ConnectionPool(host="queue", port=6379, db=0)
r = redis.Redis(connection_pool=pool)
p = r.pubsub()

p.subscribe("Log")

p.get_message()
time.sleep(0.5)

In [7]:
def getNextInQueue(p) :
    ### 
    # Get incoming items in the queue as a generator
    ###
    for mes in p.listen() :
        if type(mes["data"]) == type(b'') :
            yield pickle.loads(mes["data"])

In [8]:
# SQL

def sqlGetUser(id) :
    ###
    # Get the sql query to create a new user
    ###
    return "SELECT * FROM users WHERE users.extensionID = '{}';".format(id)

def sqlAddDiff(id, diff, date) :
    ###
    # Get the sql query to create a new diff
    ###
    return "INSERT INTO diffs VALUES ({0}, '{3}', '{1}', '{2}')".format(id, json.dumps({"+": diff["+"]}), json.dumps({"-": diff["-"]}), date)

def sqlUpdateUser(id, newCurrent) :
    ###
    # Get the sql query to update the current lists for user with users.id = id  
    ###
    return "UPDATE users SET current = '{0}' WHERE users.id = {1};".format(
            json.dumps({"current": newCurrent}),
            id
        )

def sqlCreateUser(id, current) :
    return "INSERT INTO users(extensionID, current) VALUES ('{0}', '{1}')".format(id, json.dumps({"current": current}))


In [9]:
for item in getNextInQueue(p) :
    # Get corresponding user
    postgres.execute(sqlGetUser(item[0]))
    user = postgres.fetchone()
    date = str(datetime.now())

    if user != None :
        # user already exists
        if user[-1]["current"] == item[1] :
            continue

        diff = {
            "+" : [l for l in item[1] if l not in user[-1]["current"]],
            "-" : [l for l in user[-1]["current"] if l not in item[1]]
        }

        postgres.execute(sqlAddDiff(user[0], diff, date))
        postgres.execute(sqlUpdateUser(user[0], item[1]))
    else :
        # create new user
        postgres.execute(sqlCreateUser(item[0], item[1]))
        postgres.execute(sqlGetUser(item[0]))
        user = postgres.fetchone()
        # Add initial diff
        postgres.execute(sqlAddDiff(user[0], {"+": item[1], "-": []}, date))

    # Commit changes to the database
    connection.commit()


    

        

{'type': 'message', 'pattern': None, 'channel': b'Log', 'data': b'\x80\x04\x95"\x00\x00\x00\x00\x00\x00\x00\x8c\textension\x94]\x94(\x8c\x02L1\x94\x8c\x02L2\x94\x8c\x02L3\x94e\x86\x94.'}
{'type': 'message', 'pattern': None, 'channel': b'Log', 'data': b'\x80\x04\x95\x11\x00\x00\x00\x00\x00\x00\x00\x8c\textension\x94]\x94\x86\x94.'}
{'type': 'message', 'pattern': None, 'channel': b'Log', 'data': b'\x80\x04\x95"\x00\x00\x00\x00\x00\x00\x00\x8c\textension\x94]\x94(\x8c\x02L1\x94\x8c\x02L2\x94\x8c\x02L3\x94e\x86\x94.'}
{'type': 'message', 'pattern': None, 'channel': b'Log', 'data': b'\x80\x04\x95\x17\x00\x00\x00\x00\x00\x00\x00\x8c\textension\x94]\x94\x8c\x02L1\x94a\x86\x94.'}
